SAS & HSE, Applied Time Series Forecasitng , Fall 2024

<font color="green"> Lesson #3: Holt, Winters, Theil-Wage models. Exponential Smoothing Models </font>

<span style="color:black; font-size: 12pt"></span>

Alexey Romanenko,
<font color="blue">alexromsput@gmail.com</font>

**Key words:**  exponential smoothing models, Holt model,Winters model, Theil-Wage Model

**Your feedback:**  please provide you feedback  <a href="https://forms.gle/EQgXEVQe9PPXUBzm6"> here </a>

In [2]:
import numpy as np
from datetime import datetime, timedelta

import pandas as pd
import math
import pandas.tseries.offsets as ofs

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
pd.options.plotting.backend = "plotly"

from utils import qualityMAPE, SimpleExponentialSmoothing, build_forecast, plot_ts_forecast
# from utils import plot_ts_forecast, plot_ts_forecast # don't forget to upload file in colab working repository
import warnings as w
# %matplotlib inline

# Holt model, Winters model, Theil-Wage model

In [3]:
# Wage data in RF
wage = pd.read_csv('https://raw.githubusercontent.com/aromanenko/ATSF/main/data/monthly-wage.csv', sep=';', index_col= 0, parse_dates=True)
wage.plot().update_layout(height=350, width=1350).show()

**Question:**
   * Which hidden components of the ts are not considered by SES?

## Holt Model

In [4]:
# Aggregate original TS by Years
wage_year = wage.resample("12MS").sum()[:-1] # cut 2017 year
wage_year.plot().update_layout(height=350, width=1350).show()

In [5]:
# SES Forecast for monthly agregated data
ALPHA = np.linspace(0.01,1,10)
ESparams = [{'alpha':alpha} for alpha in ALPHA]
FRC_WAGE_YEAR = build_forecast(h=1, ts=wage_year, alg_name =  'SimpleExponentialSmoothing', alg_title='SES'
                              ,params = ESparams, step='12MS')

# forecast accuracy
qlt_ses = pd.DataFrame(index = wage_year.columns, columns = FRC_WAGE_YEAR.keys())

ix = wage_year.loc['2010-01':'2018-01'].index
for param_cntr in sorted(qlt_ses.columns):
    frc_wage = FRC_WAGE_YEAR[param_cntr]
    qlt_ses[param_cntr],_ = qualityMAPE(wage_year.loc[ix], frc_wage.loc[ix])

# Draw forecast of the best SES algorithm
alg_name = qlt_ses[qlt_ses.columns].mean().sort_values().index[0]
plot_ts_forecast(wage_year.loc['1999-01-01':'2016-01-01'], FRC_WAGE_YEAR[alg_name].loc['1999-01-01':'2016-01-01']
               , ts_num=0, alg_title=alg_name)

test_period_start = '2000-01-01'
test_period_end = '2016-01-01'
print('MAPE: %s' % qualityMAPE(wage_year.loc[test_period_start:test_period_end], FRC_WAGE_YEAR[alg_name].loc[test_period_start:test_period_end])[0])

MAPE: Real wage    0.088797
dtype: float64



**Question:**
   * How to include trend component in Exponential Smoothing model?

**From Simple Exponential Smoothing to Holt Model**
$\varepsilon_t~-$ error component (unobserved noise)

$l_t$ $-$ changing slowly level of time series,

$\hat l_t~-$ an estimation of level

$\hat y_t~-$ forecast value for $y_t$ (made with delay = 1)

${e_t = y_t - \hat y_t }$

<table border="0">
 <tr>
    <td><b style="font-size:30px">$~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~$Simple Exponential Smoothing$~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~$
</b></td>
    <td><b style="font-size:30px">$~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~$ Holt model (additive linear trend)$~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~$</b></td>
 </tr>
 <tr>
    <td>

Time Series Model    
    $$y_{t} = l_t + \color{red}{\varepsilon_t},$$

Forecasting model:

$$ {\hat y_{t+d}} = \color{\red}{\hat l_t} $$


$$\hat{l}_{t} = \alpha\cdot y_t+ (1-\alpha)\cdot \hat{y}_t = \hat y_t + \alpha \cdot \color{red}{e_t}$$
    <td>

Time series model:
$$  y_{t} = l_t + \color{red}{b_t}  + {\varepsilon_t} $$

Forecasting formula:
    $$  \hat y_{t+d} = \hat l_t + \color{red}{\hat b_t} d $$
    <!-- where $l_t$, $\color{red}{b_t}$ --- estimations of unopbserved components of level and trend correspondently -->


$$       \hat l_t = \alpha y_t + (1-\alpha) (\hat l_{t-1} + \hat b_{t-1} ) = \color{red}{\hat y_{t} + \alpha e_t}$$

$$b_t = \beta (\hat l_{t} - \hat l_{t-1} ) + (1-\beta) \hat b_{t-1} = \color{red}{\hat b_{t-1}+ \alpha\beta e_t}.$$</td>
 </tr>
</table>



<!-- Доказательство

$$\beta (l_{t} - l_{t-1} ) + (1-\beta) b_{t-1} = b_{t-1}+ \beta(l_t-l_{t-1}-b_{t-1})= b_{t-1}+ \beta(l_t-l_{t-1}-b_{t-1}) +\alpha e_t$$ -->



In [27]:
###################### Holt Exponential Smoothing #########################
# x <array Tx1>- time series,
# h <scalar> - forecasting delay
# Params <dict> - dictionary with
#    alpha <scalar in [0,1]> - smoothing parameter
#    beta <scalar in [0,1]> - linear trend smoothing parameter
#
def myHoltExponentialSmoothing(x, h, params, freq=None):
    T = len(x)
    alpha = params['alpha']
    beta = params['beta']
    AdaptationPeriod = params['AdaptationPeriod']

    # retrieve date frequency in ts
    if freq is None:
      freq = pd.infer_freq(x.index)

    # prepare Forecast pd.Series to put forecasted values in it
    forecast = pd.Series(index = x.index.append(pd.date_range(x.index[-1], periods=h+1, freq=freq)[1:]), name = 'fcst '+x.name)

    # L = [np.NaN]*T.  # level component
    # B = [np.NaN]*T.  # trend component
    if alpha>1:
        w.warn('Alpha can not be more than 1')
        #alpha = 1
        return forecast
    if alpha<0:
        w.warn('Alpha can not be less than 0')
        #alpha = 0
        return forecast
    if beta>1:
        w.warn('beta can not be more than 1')
        #beta = 1
        return forecast
    if beta<0:
        w.warn('beta can not be less than 0')
        #beta = 0
        return forecast

    # initialization
    l= np.NaN
    b= np.NaN
    t0 = 0

    # forecast all ts step-by-step
    for t in range(T):
        if not math.isnan(x.iloc[t]):
            if math.isnan(l):
                l = x.iloc[t]
                b = x.iloc[t+1]-x.iloc[t]
                # t0 = np.NaN

            l_prev = l

            # initialization using Adaptation period approach
            if (t-t0+1)<AdaptationPeriod:
                l = (1-(1-alpha)*(t-t0+1)/(AdaptationPeriod))* x.iloc[t] + (1-alpha)*(t-t0+1)/(AdaptationPeriod)*(l+b)
                b = (1-beta)*(t-t0+1)/(AdaptationPeriod)*(l - l_prev) + (1-alpha)*(t-t0+1)/(AdaptationPeriod)*b
            else:
              l = alpha* x.iloc[t] + (1-alpha)*(l+b)
              b = beta* (l - l_prev) + (1- beta)*b

                # b = beta* (x[t] - l_prev) + (1- beta)*b # alternative
        # L[t] = l
        # B[t] = b
        forecast.iloc[t+h] = l+ b*h
    return forecast #, L, B

In [28]:
def mybuild_forecast(h, ts, alg_name, alg_title, params, step=None):
  'grid'

  FRC_TS = dict()

  for p in params:
      frc_ts = pd.DataFrame(columns = ts.columns)

      for cntr in ts.columns:
          frc_ts[cntr] = eval(alg_name)(ts[cntr], h, p, step)

#         frc_ts.columns = frc_ts.columns+('%s %s' % (alg_title, p))
      FRC_TS['%s %s' % (alg_title, p)] = frc_ts

  return FRC_TS

In [31]:
# SES Forecast for monthly agregated data
ALPHA = np.linspace(0.9,1,10)
BETA = np.linspace(0.0001,0.001,10)

holt_params = [{'alpha':alpha, 'beta':beta, 'AdaptationPeriod':10} for alpha in ALPHA for beta in BETA]
FRC_WAGE_YEAR = mybuild_forecast(h=1, ts=wage_year, alg_name =  'myHoltExponentialSmoothing', alg_title='Holt'
                              ,params = holt_params, step='12MS')

# forecast accuracy
qlt_holt = pd.DataFrame(index = wage_year.columns, columns = FRC_WAGE_YEAR.keys())

ix = wage_year.loc['2000-01':'2018-01'].index
for param_cntr in sorted(qlt_holt.columns):
    frc_wage = FRC_WAGE_YEAR[param_cntr]
    qlt_holt[param_cntr],_ = qualityMAPE(wage_year.loc[ix], frc_wage.loc[ix])

# # Draw forecast of the best SES algorithm
alg_name = qlt_holt[qlt_holt.columns].mean().sort_values().index[0]
plot_ts_forecast(wage_year, FRC_WAGE_YEAR[alg_name] #.loc['1999-01-01':'2018-01-01']
               , ts_num=0, alg_title=alg_name)

print('MAPE: %s' % qualityMAPE(wage_year.loc[test_period_start:test_period_end], FRC_WAGE_YEAR[alg_name].loc[test_period_start:test_period_end])[0])

MAPE: Real wage                                                               NaN
Real wage; Holt {'alpha': 1.0, 'beta': 0.001, 'AdaptationPeriod': 10}   NaN
dtype: float64


In [32]:
# Find the best model
qlt_holt.transpose().sort_values(by = 'Real wage')[:11]

,Real wage
"Holt {'alpha': 1.0, 'beta': 0.001, 'AdaptationPeriod': 10}",0.059509
"Holt {'alpha': 1.0, 'beta': 0.0009, 'AdaptationPeriod': 10}",0.059510
"Holt {'alpha': 1.0, 'beta': 0.0007999999999999999, 'AdaptationPeriod': 10}",0.059511
"Holt {'alpha': 1.0, 'beta': 0.0007, 'AdaptationPeriod': 10}",0.059512
"Holt {'alpha': 1.0, 'beta': 0.0006000000000000001, 'AdaptationPeriod': 10}",0.059512
"Holt {'alpha': 1.0, 'beta': 0.0005, 'AdaptationPeriod': 10}",0.059513
"Holt {'alpha': 1.0, 'beta': 0.00039999999999999996, 'AdaptationPeriod': 10}",0.059514
"Holt {'alpha': 1.0, 'beta': 0.0003, 'AdaptationPeriod': 10}",0.059515
"Holt {'alpha': 1.0, 'beta': 0.00019999999999999998, 'AdaptationPeriod': 10}",0.059515
"Holt {'alpha': 1.0, 'beta': 0.0001, 'AdaptationPeriod': 10}",0.059516


## Winters Model

In [44]:
ts = pd.read_csv('https://raw.githubusercontent.com/aromanenko/ATSF/main/data/seasonal_ts.csv', parse_dates=['Dates'], sep=';', dayfirst=True, index_col='Dates')
ts.index = pd.to_datetime(ts.index)
ts.index.names=['Timestamp']
ts = ts.sort_index() # sort index
ts.head()

,Item1,Item2,Item3,Item4,Item5,Item6,Item7,Item8,Item9,Item10
Timestamp,,,,,,,,,,
2006-01-01,0,49,64,70.468,0,45.182,71,21.664,138,0
2006-01-02,0,56,60,57.368,0,39.506,27,19.664,76,6
2006-01-03,0,61,71,34.350,0,28.064,10,10.402,152,12
2006-01-04,0,32,59,40.186,0,40.256,27,9.938,67,12
2006-01-05,0,45,61,28.914,0,35.784,27,16.672,49,24


In [45]:
# Interval of ts
ts.loc['2007-01-01':'2007-01-05']

,Item1,Item2,Item3,Item4,Item5,Item6,Item7,Item8,Item9,Item10
Timestamp,,,,,,,,,,
2007-01-01,156,135,64,95.925,68,57.934,30,28.854,19,0
2007-01-02,138,117,62,43.775,63,52.693,19,24.478,14,18
2007-01-03,148,98,67,50.750,57,43.406,15,15.704,18,6
2007-01-04,147,86,67,54.020,47,53.018,9,21.846,15,6
2007-01-05,146,124,71,50.400,59,74.212,15,33.082,2,12


In [46]:
# fig = plt.figure()
ts.loc['2005-07-01':'2007-12-31', ts.columns[range(3)]].plot().update_layout(height=350, width=1350).show()
# to save the pictures
# plt.savefig('../Lecture_TS_Forecasting/pic/TS_Example.eps', bbox_inches='tight', pad_inches=0, format='eps', dpi=1000)

**Questions**

 - What are key aspects of these retail ts?
 - How to change SES model to consider seasonlaity in original data?

**Additive Winters Model = Additive Seasonality**

Time Series model
$$y_t = l_t + s_{t}+\varepsilon_t$$
where $s_t$ - is a seasonal component of period $p$

Forecasting Model
$$\hat{y}_{t+d} = \hat l_t +  \hat s_{t+(d\mod p) - p}; \\
		\hat l_{t}       	=  \alpha \left(y_t - \hat s_{t-p}\right)+ \left(1-\alpha\right) \left(\hat l_{t-1}\right)=\color{red}{\hat l_{t-1} + \alpha e_t}; \\
		\hat s_t         	= \gamma\left(y_t- \hat l_{t}\right) + \left(1-\gamma\right)\hat s_{t-p} = \color{red}{\hat s_{t-p} + \gamma(1-\alpha)e_t}.
		$$

In [38]:

###################### Additive Winters Exponential Smoothing #########################
# x <array Tx1>- time series,
# h <scalar> - forecasting delay
# Params <dict> - dictionary with
#    alpha <scalar in [0,1]> - level smoothing parameter
#    gamma <scalar in [0,1]> - seasonality smoothing parameter

def myAdditiveWintersExponentialSmoothing(x, h, params, freq = None):
    T = len(x)
    alpha = params['alpha']
    gamma = params['gamma']
    p = params['seasonality_period']

    # retrieve date frequency in ts
    if freq is None:
      freq = pd.infer_freq(x.index)

    # prepare Forecast pd.Series to put forecasted values in it
    forecast = pd.Series(index = x.index.append(pd.date_range(x.index[-1], periods=h+1, freq=freq)[1:]), name = 'fcst '+x.name)

    l= np.NaN
    s= []

    for cntr in range(T):
        if not math.isnan(x.iloc[cntr]):
            if math.isnan(l):
                l= x.iloc[cntr]
            if len(s)==0:
                # looking in the future
                for i in range(p):
                    s.append(x.iloc[i])
            if cntr<p:
                l = alpha*(x.iloc[cntr]-s[cntr])+(1-alpha)*l # recurrent smoothing of level
            else:
                l = alpha*(x.iloc[cntr]-s[cntr-p])+(1-alpha)*l # recurrent smoothing of level
                s.append(gamma*(x.iloc[cntr]-l)+(1-gamma)*s[cntr-p])

        forecast.iloc[cntr+h] = l + s[cntr+h-(1+h//p)*p]
    return forecast


In [47]:
# SES Forecast for monthly agregated data
ALPHA = np.linspace(0.01,1,10)
GAMMA = np.linspace(0.01,1,10)

wint_params = [{'alpha':alpha, 'gamma':gamma, 'seasonality_period':7} for alpha in ALPHA for gamma in GAMMA]
FRC_SEAS_TS = mybuild_forecast(h=1, ts=ts, alg_name =  'myAdditiveWintersExponentialSmoothing', alg_title='AWinters'\
                              ,params = wint_params, step='D')

# # forecast accuracy
qlt_winters = pd.DataFrame(index = ts.columns, columns = FRC_SEAS_TS.keys())

ix = ts.loc['2006-01-01':'2007-01-01'].index
for param_cntr in sorted(qlt_winters.columns):
    frc_wage = FRC_SEAS_TS[param_cntr]
    qlt_winters[param_cntr],_ = qualityMAPE(ts.loc[ix], frc_wage.loc[ix])

# # Draw forecast of the best algorithm (mean on all ts)
alg_name = qlt_winters[qlt_winters.columns].mean().sort_values().index[0]
plot_ts_forecast(ts.loc[:], FRC_SEAS_TS[alg_name].loc[:]
               , ts_num=0, alg_title=alg_name)

test_period_start = '2007-01-01'
test_period_end = '2007-06-01'
print('MAPE: %s' % qualityMAPE(ts.loc[test_period_start:test_period_end], FRC_SEAS_TS[alg_name].loc[test_period_start:test_period_end])[0])

MAPE: Item1     0.133644
Item2     0.269940
Item3     0.178748
Item4     0.173496
Item5     0.219208
Item6     0.266236
Item7     0.328734
Item8     0.300761
Item9     0.951519
Item10    0.669742
dtype: float64


In [50]:
plot_ts_forecast(ts.loc['1999-01-01':'2018-01-01'], FRC_SEAS_TS[alg_name].loc['1999-01-01':'2018-01-01']
               , ts_num=5, alg_title=alg_name)

**Questions**
  - Does is it seem that forecasting mdoel is proper?

In [51]:
# the best algorithm for Item9
alg_name = qlt_winters.loc[qlt_winters.index[8]].sort_values().index[0]
plot_ts_forecast(ts, FRC_SEAS_TS[alg_name], ts_num=8, alg_title=alg_name)

## Theil-Wage Model

**Questions**
  - How to consider both additive trend and additive seasonality?

Time Series Model:
$$ y_t = l_t + b_t + s_t + \varepsilon_t. $$
$s_t$ - seasonal component of period $p$,

Forecasting Model
$$ \hat y_{t+d} = \hat l_t + \hat b_t\cdot d + \hat s_{t+(d\mod p)-p}. $$

$$ \hat l_t = \alpha (y_t - \hat s_{t-p}) + (1-\alpha) (\hat l_{t-1} + \hat b_{t-1} )=\color{red}{\hat l_{t-1} + \hat b_{t-1} + \alpha e_t};$$

$$\hat b_t = \beta (\hat l_{t} - \hat l_{t-1} ) + (1-\beta) \hat b_{t-1} = \color{red}{\hat b_{t-1} + \alpha\beta e_t};$$

$$ \hat s_t =\gamma (y_t-\hat l_{t}) + (1-\gamma) \hat s_{t-p} = \color{red}{\hat s_{t-p} + \gamma(1-\alpha)e_t}.$$

Note: there can be other approaches of calculating hidden components, example:
$$ \hat s_t =\gamma (y_t-\color{green}{\hat l_{t-1}-\hat b_{t-1}}) + (1-\gamma) \hat s_{t-p} = \hat s_{t-p} + \color{green}{\gamma e_t}.$$

In [54]:

###################### Winters Exponential Smoothing #########################
# x <array Tx1>- time series,
# h <scalar> - forecasting delay
# Params <dict> - dictionary with
#    alpha <scalar in [0,1]> - level smoothing parameter
#    beta <scalar in [0,1]> - trend smoothing parameter
#    gamma <scalar in [0,1]> - seasonality smoothing parameter

def myTheilWageExponentialSmoothing(x, h, params, freq = None):
    T = len(x)
    alpha = params['alpha']
    beta = params['beta']
    gamma = params['gamma']
    p = params['seasonality_period']

    # retrieve date frequency in ts
    if freq is None:
      freq = pd.infer_freq(x.index)

    # prepare Forecast pd.Series to put forecasted values in it
    forecast = pd.Series(index = x.index.append(pd.date_range(x.index[-1], periods=h+1, freq=freq)[1:]), name = 'fcst '+x.name)

    l= np.NaN
    b=np.NaN
    s= []

    for cntr in range(T):
        if not math.isnan(x.iloc[cntr]):
            if math.isnan(l):
                l= x.iloc[cntr]
            if math.isnan(b):
                b= 0

            if len(s)==0:
                for i in range(p):
                    s.append(x.iloc[i])

            if cntr<p:
                l_old=l
                l = alpha*(x.iloc[cntr]-s[cntr])+(1-alpha)*(l+b)
                b=beta*(l-l_old)+(1-beta)*b
            else:
                l_old=l
                l = alpha*(x.iloc[cntr]-s[cntr-p])+(1-alpha)*(l+b) # recurrent smoothing of level
                b=beta*(l-l_old)+(1-beta)*b
                s.append(gamma*(x.iloc[cntr]-l)+(1-gamma)*s[cntr-p])

        forecast.iloc[cntr+h] = l+b*h + s[cntr+h - (1+h//p)*p]
    return forecast

In [56]:
# TW Forecast for monthly agregated data
ALPHA = np.linspace(0.01,1,10)
BETA = np.linspace(0.01,1,10)
GAMMA = np.linspace(0.01,1,10)

tw_params = [{'alpha':alpha, 'beta':beta, 'gamma':gamma, 'seasonality_period':12} for alpha in ALPHA for gamma in GAMMA for beta in BETA]
FRC_TREND_SEAS_TS = mybuild_forecast(h=12, ts=wage, alg_name = 'myTheilWageExponentialSmoothing', alg_title='TW'\
                              ,params = tw_params, step='ME')

# # forecast accuracy
qlt_tw = pd.DataFrame(index = wage.columns, columns = FRC_TREND_SEAS_TS.keys())

ix = wage.loc['2010-01':'2018-01'].index
for param_cntr in sorted(qlt_tw.columns):
    frc = FRC_TREND_SEAS_TS[param_cntr]
    qlt_tw[param_cntr],_ = qualityMAPE(wage.loc[ix], frc.loc[ix])

# # Draw forecast of the best SES algorithm
alg_name = qlt_tw[qlt_tw.columns].mean().sort_values().index[0]
plot_ts_forecast(wage.loc['1999-01-01':'2018-01-01'], FRC_TREND_SEAS_TS[alg_name].loc['1999-01-01':'2018-01-01']
               , ts_num=0, alg_title=alg_name)

test_period_start = '2015-01-01'
test_period_end = '2018-01-01'
print('MAPE: %s' % qualityMAPE(wage.loc[test_period_start:test_period_end], FRC_TREND_SEAS_TS[alg_name].loc[test_period_start:test_period_end])[0])

MAPE: Real wage    0.068202
dtype: float64


# Exponential Smoothing Models Family

<!-- # use only for the first run:
# !pip install sktime
from sktime.datasets import load_airline
y = load_airline()
y_train, y_test = temporal_train_test_split(y)
fh = ForecastingHorizon(y_test.index, is_relative=False)
forecaster = ThetaForecaster(sp=12)  # monthly seasonal periodicity
forecaster.fit(y_train)
y_pred = forecaster.predict(fh)
mean_absolute_percentage_error(y_test, y_pred)
y.plot(figsize = (25,10))
y_pred.plot() -->

## Multiplicative trend model
**Multiplicative (exponential) trend:**
		$$
		\hat{y}_{t+d|t} = l_tb_t^d, \\
		l_{t}       = \alpha y_t + \left(1-\alpha\right) \left(l_{t-1} b_{t-1}\right), \\
		b_t         = \beta \frac{l_t}{l_{t-1}} + \left(1-\beta\right) b_{t-1}.
		$$





## Damp-trend model

**Additive damped trend**
		$$
		\hat{y}_{t+d|t} = l_t + \left(\phi + \phi^2 + \dots + \phi^{d}\right) b_t, \\
		l_{t}       = \alpha y_t + \left(1-\alpha\right) \left(l_{t-1} +\phi b_{t-1}\right), \\
		b_t         = \beta \left(l_t - l_{t-1}\right) + \left(1-\beta\right)\phi b_{t-1}.
		$$

<br></br>
**Multiplicative damped trend**
		$$
		\hat{y}_{t+d|t} = l_t b_t^{\left(\phi + \phi^2 + \dots + \phi^{d}\right)}, \\
		l_{t}       = \alpha y_t + \left(1-\alpha\right) l_{t-1} b_{t-1}^{\phi}, \\
		b_t         = \beta\frac{l_t}{l_{t-1}} + \left(1-\beta\right)b_{t-1}^{\phi}.
		$$


## Multiplicative Seasonality or Multiplicative Trend

**Additive Trend and Multiplicative Seasonality**
$$ \hat y_{t+d} = (l_t + b_t d) \cdot s_{t+d},$$
    
$$   l_t = \alpha (y_t / s_{t-p}) + (1-\alpha) (l_{t-1} + b_{t-1} ) = \color{red}{l_{t-1} + b_{t-1} + \alpha e_t/ s_{t-p}}; $$


$$  b_t = \beta (l_{t} - l_{t-1} ) + (1-\beta) b_{t-1} = \color{red}{b_{t-1}+ \alpha\beta e_t/s_{t-p}}; $$


$$s_t = \gamma (y_t/l_t) + (1-\gamma) s_{t-p} = \color{red}{s_{t-p} + \gamma (1-\alpha) e_t/l_t} .   $$

<br></br>
**Multiplicative Trend and Additive Seasonality**
$$ \hat y_{t+d} = (l_t \cdot b_t d) + s_{t+d},$$


$$ l_t = \alpha (y_t - s_{t-p}) + (1-\alpha) (l_{t-1} \cdot b_{t-1} );$$

$$b_t = \beta (l_{t} / l_{t-1} ) + (1-\beta) b_{t-1};$$

$$ s_t =\gamma (y_t-l_t) + (1-\gamma) s_{t-p}.$$

<br></br>
**Multiplicative Trend and Mupltiplicative Seasonlality**

$$    \hat y_{t+d} = l_t (b_t)^d \cdot s_{t + (d \bmod p) -p},    $$

$$    l_t \mathop{=} \alpha (y_t / s_{t-p}) + (1-\alpha) l_{t-1} b_{t-1} = \color{red}{l_{t-1} b_{t-1} + \alpha e_t/s_{t-1}};$$

$$ b_t \mathop{=} \beta (l_{t} / l_{t-1} ) + (1-\beta) b_{t-1} =  \color{red}{b_{t-1}+\alpha\beta e_t/s_{t-1}};$$

$$ s_t \mathop{=} \gamma (y_t/l_t) + (1-\gamma) s_{t-p} = \color{red}{s_{t-p} + \gamma(1-\alpha)e_t/l_{t}}.$$


## Taxonomy of Time Series Models

 <table border="0">
 <tr>
    <td>
     Trend\Seasonality
    </td>
    <td>
       Seasonality N (None)
    </td>
    <td>
       Seasonality A (Additive)
    </td>
    <td>
       Seasonality M (Multiplicative)
    </td>
</tr>
 <tr>
    <td>
      Trend N (None)
    </td>
    <td>(N, N)</td>
    <td>(N, A)</td>
    <td>(N, M)</td>
 </tr>
 <tr>
    <td>
      Trend A (Additive)
    </td>
    <td>(A, N)</td>
    <td>(A, A)</td>
    <td>(A, M)</td>
 </tr>
 <tr>
    <td>
      Trend Ad (Additive damped)
    </td>
    <td>(Ad, N)</td>
    <td>(Ad, A)</td>
    <td>(Ad, M)</td>
 </tr>
 <tr>
    <td>
      Trend M (Multiplicative)
    </td>
    <td>(M, N)</td>
    <td>(M, A)</td>
    <td>(M, M)</td>
 </tr>
 <tr>
    <td>
      Trend Md (Multiplicative damped)
    </td>
    <td>(Md, N)</td>
    <td>(Md, A)</td>
    <td>(Md, M)</td>
 </tr>
</table>

## Pro&Cons Exponential Smoothing models
   PRO | Cons
-------------------|------------------
Easy to Interpretate       | Initialization issues
Very fast, low computational time       |  Are not stable (retrain is needed)
Can be applied to ts with missings       | Is not possible to inlcude causal variables

<!-- # Use Case: Working with ESM models in SAS Visual Forecasting tool -->
 <!-- - More than 50% of all TS are forecasted by models from ES family -->


# HW1
see https://github.com/aromanenko/ATSF/blob/main/HW1.ipynb

# Chech Questions
  * Write down formula of Holt model forecast
  * Write down formula of Winters model forecast
  *
  

# Materials

- [Лукашин Ю.П. Адаптивные методы краткосрочного прогнозирования временных рядов. Финансы и статистика. 2003, главы 1,4,5,7.](https://disk.yandex.ru/i/9k3i4SHFSgPP_A)

- [Магнус Я.Р., Катышев П.К., Пересецкий А.А. Эконометрика. Начальный курс., глава 11](https://disk.yandex.ru/i/gkzWBBCv42ZcBA)

- [Rob J Hyndman, Forecasting: Principles & Practice, 23-25 September 2014](https://robjhyndman.com/uwafiles/fpp-notes.pdf)

- [James D Hamilton, Time Series Analysis, 1994](http://mayoral.iae-csic.org/timeseries2021/hamilton.pdf)